In [1]:
import os
#!pip3 install tabulate
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
from IPython.display import display, HTML
import multiprocessing as mp
from multiprocessing import Pool
import h5py

import pickle
pickle.HIGHEST_PROTOCOL = 4

## The iput data is a set of  BIO files of a dataset

In [ ]:
dataset = ['training', 'testing', 'developing']
path    = os.getcwd() + '/BIO_data/'

#print(path)

for folder in dataset:
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path + folder):
        for file in f:
            if file.endswith(".bio"):
                #print(os.path.join(r, file))
                files.append(os.path.join(r, file))
    
    data = ""
    for i, f in enumerate(files):
        with open(f) as fp: 
            data += fp.read()
        data += '\n'
    
    with open (path + folder + '.bio', 'w') as fp:
        fp.write(data)

## Develop Proccess to Covert to dataframe

In [ ]:
fileinput = ['training.bio', 'testing.bio', 'developing.bio']
columnas  = ["word", "ner", "lema", "postag", "tokpos"]

In [ ]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [ ]:
def open_file(filename):
    file = open(filename, encoding="utf-8")
    return file

In [ ]:
def parse_chunk(data):
    results = []
    flat_eos = 0
    for p, line in enumerate(data):
        try:
            parsed = dict()
            if len(line) == 0:
                flat_eos = 0
            elif len(line) == 4:
                for k, h in enumerate(columnas):
                    if k < 4:
                        parsed[h] = line[k]
                    elif k == 4 and flat_eos == 0:
                        parsed["tokpos"] = 'SOS'
                        flat_eos = 1
                    else:
                        parsed["tokpos"] = 'WOS'
                        flat_eos = 1
            results.append(parsed)
        except Exception as e:
            print(e, p)
    return results

In [ ]:
def run_process(file):
    file = open_file(file).readlines()
    listify = [line.split() for line in file]

    data = chunks(listify, int(len(listify) / (mp.cpu_count())))
    p = Pool(processes=mp.cpu_count())
    results = [p.apply_async(parse_chunk, args=(list(x),)) for x in data]

    # wait for results
    results = [item.get() for item in results]
    results = sum(results, [])
    
    return results

In [ ]:
k1 = run_process(path + fileinput[0])

df1 = pd.DataFrame(k1, columns=columnas)

print(len(df1))
#df1 = df1[df1['word'].notna()]
#print(df1.loc[13, 'word'])

df1.dropna(subset = ["word"], inplace= True)

print(len(df1))

df1.reset_index(drop=True, inplace=True)

df1.to_hdf(fileinput[0] + '.h5', key='df1', mode='w')

## Develop Process of Conversion DataSet

In [ ]:
name1 = "training.bio"
dta1 = pd.read_hdf(name1 + '.h5', 'df1')
df1 = dta1[['word', 'ner', 'tokpos']]
indi1 = df1.loc[df1['tokpos'] == 'SOS']

name2 = "testing.bio"
dta2 = pd.read_hdf(name2 + '.h5', 'df2')
df2 = dta2[['word', 'ner', 'tokpos']]
indi2 = df2.loc[df2['tokpos'] == 'SOS']

name3 = "developing.bio"
dta3 = pd.read_hdf(name3 + '.h5', 'df3')
df3 = dta3[['word', 'ner', 'tokpos']]
indi3 = df3.loc[df3['tokpos'] == 'SOS']

In [ ]:
for index, row in indi1.iterrows():
    if index != 0:
        dta1.at[(index - 1), "tokpos"] = 'EOS'

In [ ]:
for index, row in indi2.iterrows():
    if index != 0:
        dta2.at[(index - 1), "tokpos"] = 'EOS'

In [ ]:
for index, row in indi3.iterrows():
    if index != 0:
        dta3.at[(index - 1), "tokpos"] = 'EOS'

In [ ]:
dta1.to_hdf(name1 + '.mod.h5', key='df1', mode='w')

dta2.to_hdf(name2 + '.mod.h5', key='df2', mode='w')

dta3.to_hdf(name3 + '.mod.h5', key='df3', mode='w')